# pySSV GUI Library Examples
*pySSV* includes a powerful and easy to use immediate mode GUI library to create interactive user interfaces for your shaders.

Check out the documentation for more details about the GUI library: [SSVGUI Documentation](https://pyssv.readthedocs.io/en/latest/reference.html#gui-library)

In [ ]:
# Google colab support
try:
    # Try enabling custom widgets, this will fail silently if we're not in Google Colab
    from google.colab import output
    output.enable_custom_widget_manager()
    # Install pySSV for this session
    %pip install pySSV
except:
    pass

In [ ]:
import pySSV as ssv
from pySSV.ssv_gui import create_gui, SSVGUI
from pySSV import ssv_colour

class MyGUI:
    # We create a new class for the GUI so that state can be stored between on_gui_draw calls
    checkbox_value = False
    slider_font_size = 10.
    slider_font_weight = 0.4
    slider_rounding = 4
    slider_int = 0
    slider_non_lin = 0
    
    def on_gui_draw(self, gui: SSVGUI):
        # Begin drawing our GUI in here.
        # This is an immediate mode gui, so any time the GUI is invalidated (on mouse click for instance), we redraw the entire GUI.
        # The GUI drawing methods used here should *only* be used within the on_gui callback.
        gui.begin_vertical(pad=True)
        gui.rounded_rect(ssv_colour.ui_base_bg, overlay_last=True)
        gui.button("Hello world!")
        gui.button("GUIs are Awesome", ssv_colour.orange)
        
        gui.begin_horizontal()
        gui.rounded_rect(ssv_colour.violet)
        gui.rounded_rect(ssv_colour.darkgrey)
        gui.end_horizontal()
        
        gui.label("Shaders are cool!", font_size=float(self.slider_font_size), x_offset=5, shadow=True, overlay_last=True)

        gui.begin_horizontal()
        gui.button("Horizontal", ssv_colour.red)
        gui.button("Layout", ssv_colour.green)
        gui.button("Groups", ssv_colour.blue)
        gui.end_horizontal()

        gui.space()
        self.checkbox_value = gui.checkbox("We can enable layout groups", self.checkbox_value)
        gui.begin_toggle(self.checkbox_value)
        gui.button("This control can be hidden", ssv_colour.olive)
        gui.button("Peekaboo", ssv_colour.emeraldgreen)
        gui.end_toggle()
        gui.button("Controls are automatically layed out", ssv_colour.navy)
        gui.label("Text can be styled", font_size=22., x_offset=5, colour=ssv_colour.orange)
        gui.label("Text can be styled", font_size=30., x_offset=5, colour=ssv_colour.white, weight=float(self.slider_font_weight))
        gui.label("Text can be styled", font_size=30., x_offset=5, colour=ssv_colour.limegreen, weight=0.75)
        gui.label("Text can be styled", font_size=16., x_offset=5, colour=ssv_colour.cyan, italic=True, shadow=True)

        self.slider_font_size = gui.slider("Sliders!", self.slider_font_size, min_value=0, max_value=100, step_size=1, power=1., track_thickness=4)
        self.slider_font_weight = gui.slider("Font weight", self.slider_font_weight, min_value=0.25, max_value=0.8)
        self.slider_rounding = gui.slider(f"Corner radius = {float(self.slider_rounding):.2f}", self.slider_rounding, min_value=0, max_value=20)
        self.slider_int = gui.slider(f"Integer slider = {self.slider_int}", self.slider_int, min_value=0, max_value=100, step_size=20, track_thickness=10)
        self.slider_non_lin = gui.slider(f"Nonlinear slider = {float(self.slider_non_lin):.2f}", self.slider_non_lin, min_value=0, max_value=100, power=2., track_thickness=10)
        gui.rounding_radius = self.slider_rounding.result
        
        gui.space(height=30)
        
        gui.end_vertical()

# Set up a basic canvas
canvas = ssv.canvas(use_renderdoc=True)
# Now create an SSVGUI
gui = create_gui(canvas)
my_gui = MyGUI()
# Register a callback to the on_gui event
gui.on_gui(lambda x: my_gui.on_gui_draw(x))
# Now we render a shader as usual
canvas.shader("""
#pragma SSV shadertoy
void mainImage(out vec4 fragColor, in vec2 fragCoord)
{
    // Normalized pixel coordinates (from 0 to 1)
    vec2 uv = fragCoord/iResolution.yy;
    // Colour changing over time
    vec3 col = sin(uv.xyx + iTime * vec3(3, 4, 5)) * 0.5 + 0.5;
    float alpha = smoothstep(0.1, 0.1+2./iResolution.y, 1.-length(uv*2.-1.));
    // Output to screen
    fragColor = vec4(vec3(col), 1.0);
}
""")
canvas.run(stream_quality=100)